# Comparison for LogSumExp

The LogSumExp-problem with linear consraints:
	$$\log(1 + \sum\limits_{k=1}^n \exp(\alpha x_k)) + \frac{\beta}{2} \|\textbf{x}\|_2^2 \rightarrow \min_{\textbf{x}\in \mathbb{R}^N}$$
	$$\text{s.t.} (\textbf{b}_j, \textbf{x}) - c_j <= 0, j = 1\dots M$$
	
This problem can be converted into saddle point problem:
	$$\min_{\textbf{x}} \max_{\textbf{y}\in\mathbb{R}^M_+} S(\textbf{x},\textbf{y}),$$
where $S(\textbf{x}, \textbf{y}) = r(\textbf{x})+F(\textbf{x}, \textbf{y}) - h(\textbf{y})$
	$$r(\textbf{x}) = \log(1 + \sum\limits_{k=1}^n \exp(\alpha x_k)) + \frac{\beta}{2} \|\textbf{x}\|_2^2$$
	$$F(\textbf{x}, \textbf{y}) = \sum\limits_{j=1}^M x_j(\textbf{b}_j, \textbf{y}) = (\textbf{x}, B\textbf{y})$$
	$$h(\textbf{y}) = (\textbf{c}, \textbf{y})$$

So N is dimensional of external variable, M - internal. But we will consider the external problem as help problem.

In [1]:
import numpy as np
from TestFunctions import LogSumExp_difalpha as LogSumExp
from TestFunctions.TestFunctions import TestFunction
from Solvers import GradientMethods
from Solvers.HalvCube import Dichotomy_exact
from Solvers.GradientMethods import FGM_internal
from Solvers.Ellipsoids import ellipsoid
import matplotlib.pyplot as plt
%matplotlib notebook
import comparison
import imp

from comparison import method_comparison, create_methods_dict

In [64]:
def get_LSE(N = 100, M = 2, alpha = 0.001, beta = 0.5e-3, k = 1, eps = 1e-3):
    """
    The function is to get class of LSE. All parameters mathes the
    parameters above.
    
    N is dimensional of primal problem
    M is number of constraints. It is also the dual dimension
    alpha is parameter of LogSumExp problem
    beta is coefficient of l2-regularization for primal problem
    
    k is parameter for to generate matrix B
    """
    np.random.seed(N*M//13)
    alpha_ = np.random.uniform(size = (N,), low = -alpha, high = alpha)
    alpha = np.max(np.abs(alpha_))
    B = []
    for i in range(M):
        b = np.random.uniform(low = -k, high = 0, size = (N,)) * np.sign(alpha_)
        p = 0.1
        z = np.array(np.random.uniform(low = 0, high = 1, size = (N,))<=p, dtype = np.float32)
        B.append(b * z)
    B = np.array(B)
    #B = np.random.uniform(low = -k, high = 0, size = (M, N))
    c = np.ones((M,))
    x_ = np.zeros((N,))
    f_ = LogSumExp.r(alpha, beta).get_value(x_)
    gamma = c.max()
    y_max = f_/gamma
    Q = [[0, y_max] for i in range(M)]
    Q = np.array(Q)
    size_domain = np.linalg.norm(Q[:,0]-Q[:,1])
    solver = FGM_internal
    Q = Q.copy()
    LSE_y_external = TestFunction(r = LogSumExp.h(c, size_domain = size_domain),
                    F = LogSumExp.F_tilde(B, size_domain),
                    h = LogSumExp.r(alpha_, beta, size_domain),
                    solver = solver, 
                    get_start_point = lambda x: (1/beta * -x.dot(B), alpha * np.sqrt(N)/beta))
    solver = ellipsoid
    Q = Q.copy()
    beta_ = eps/ size_domain * 2
    LSE_ellipsoid = TestFunction(r = LogSumExp.r(alpha_, beta, size_domain),
                    F = LogSumExp.F(B, size_domain),
                    h = LogSumExp.h(c, beta = beta_, size_domain = size_domain),
                    solver = solver, 
                    get_start_point = lambda x: ((Q[:,0] + Q[:, 1])/2, size_domain/2))
    solver = lambda func, grad, L, mu, start_point, cond: Dichotomy_exact().Halving(func, 
                                                                                    grad, 
                                                                                    L, mu, 
                                                                                    eps, 
                                                                                    start_point,
                                                                                    cond)
    Q = Q.copy()
    LSE_dichotomy = TestFunction(r = LogSumExp.r(alpha_, beta, size_domain),
                F = LogSumExp.F(B, size_domain),
                h = LogSumExp.h(c, beta = beta_, size_domain = size_domain),
                solver = solver, 
                get_start_point = lambda x: ((Q[:,0] + Q[:, 1])/2, size_domain/2))
    solver = FGM_internal
    Q = Q.copy()
    LSE_fgm = TestFunction(r = LogSumExp.r(alpha_, beta, size_domain),
                F = LogSumExp.F(B, size_domain),
                h = LogSumExp.h(c, beta = beta_, size_domain = size_domain),
                solver = solver, 
                get_start_point = lambda x: ((Q[:,0] + Q[:, 1])/2, size_domain/2))
    R = alpha * np.sqrt(N)/beta
    return (LSE_y_external, Q), (LSE_ellipsoid, R), (LSE_dichotomy, R), (LSE_fgm, R), (alpha_, B, c, beta)

In [65]:
def start_comparison(N = 100, M = 2, alpha = 0.001, beta = 0.5e-3, k = 1000, eps = 1e-3, time_max = 0.2):
    LSE_res = get_LSE(N = N, M = M, alpha = alpha, beta = beta, k = k, eps = eps)
    (LSE_y_external, Q), (LSE_ellipsoid, R), (LSE_dichotomy, R), (LSE_fgm, R), stop_cond_args = LSE_res
    keys = {"FGM":"y, x - FGM+FGM",
        "Ellipsoids": "y, x - Ellipsoids+FGM",
        "Dichotomy": "y, x - Dichotomy+FGM"}
    history = {}
    f_dict = {}
    methods = {}
    methods = create_methods_dict(LSE_y_external, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = time_max, stop_cond_args = stop_cond_args, keys = keys)
    for key in methods:
        cond = lambda y, R: LSE_y_external.L_xx * R <= 1e-3
        f = lambda y: LSE_y_external.get_value(y, LSE_y_external.get_solution(y, cond)[0])
        f_dict[key] = f
    Q = np.vstack([np.zeros((N, )) - R * np.ones((N,)), np.zeros((N, )) + R * np.ones((N,))]).T
    methods["x, y - FGM+FGM"] = create_methods_dict(LSE_fgm, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = time_max, inverse = True,  stop_cond_args = stop_cond_args, keys = {"FGM":"x, y - FGM+FGM"})["x, y - FGM+FGM"]
    cond = lambda y, R: LSE_fgm.L_xx * R <= eps
    f = lambda y: LSE_fgm.get_value(y, LSE_fgm.get_solution(y, cond)[0])
    f_dict["x, y - FGM+FGM"] = f

    methods["x, y - FGM+Ellipsoids"] = create_methods_dict(LSE_ellipsoid, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = time_max, inverse = True,  stop_cond_args = stop_cond_args, keys = {"FGM":"x, y - FGM+Ellipsoids"})["x, y - FGM+Ellipsoids"]
    cond = lambda y, R: LSE_ellipsoid.L_yy * R <= eps
    f = lambda y: LSE_ellipsoid.get_value(y, LSE_ellipsoid.get_solution(y, cond)[0])
    f_dict["x, y - FGM+Ellipsoids"] = f
    Q = np.vstack([np.zeros((N, )) - R * np.ones((N,)), np.zeros((N, )) + R * np.ones((N,))]).T
    methods["x, y - FGM+Dichotomy"] = create_methods_dict(LSE_dichotomy, np.mean(Q, axis = 1),
                              np.linalg.norm(Q[:,0]-Q[:,1]),
                              Q, eps, history, time_max = time_max, inverse = True, stop_cond_args = stop_cond_args, keys = {"FGM":"x, y - FGM+Dichotomy"})["x, y - FGM+Dichotomy"]
    cond = lambda y, R: LSE_dichotomy.L_xx * R <= eps
    f = lambda y: LSE_dichotomy.get_value(y, LSE_dichotomy.get_solution(y, cond)[0])
    f_dict["x, y - FGM+Dichotomy"] = f
    #comparison.method_comparison(methods)
    for key in methods:
        methods[key]()
    return history, stop_cond_args

In [66]:
def parse_results(history, alpha_, B, c, beta, keys = ["Ellipsoids", "Dichotomy", "FGM"]):
    s_key = ""
    s_feas = ""
    s_time = ""
    s_value = ""
    for key in keys:
        s_key += key + "\t$"
        if "y, x" in key:
            lambda_, x = history[key][-1][0]
        else:
            x, lambda_ = history[key][-1][0]
        s_feas += str((B @ x <= c).any()) + "\t\t$"
        s_time += "%.3f"%(history[key][-1][1] - history[key][0][1]) + "\t\t$"
        x_ = alpha_ * x
        M = np.max(x_)
        x_ = x_ - M
        s_value+= "%.3f"%(M + np.log(np.exp(-M) + np.exp(x_).sum()) + beta * np.linalg.norm(x)**2) + "\t\t$"
    print(s_key)
    print(s_feas)
    print(s_time)
    print(s_value)

## M= 2

In [67]:
M = 2

### N = 100

In [68]:
N = 100

In [69]:
eps = 1e-3
history, stop_cond_args = start_comparison(N = N, M = M, alpha = 0.0001, k = 10**3, beta = 1e-4, eps = eps, time_max = 5)
alpha_, B, c, beta = stop_cond_args

In [70]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$x, y - FGM+Ellipsoids	$x, y - FGM+Dichotomy	$
False		$True		$True		$True		$False		$True		$
5.004		$0.036		$0.622		$0.052		$5.007		$3.562		$
4.615		$4.685		$4.700		$4.615		$4.615		$4.615		$


In [71]:
len(history["y, x - Dichotomy+FGM"])

27

In [72]:
len(history["y, x - Dichotomy+FGM"])

27

In [9]:
eps = 1e-6
history, stop_cond_args = start_comparison(N = N, M = M,beta = 1e-6, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

/home/elias/Desktop/PROJECTS/SPP/SPP/Solvers/Ellipsoids.py:76: RuntimeWarning: divide by zero encountered in true_divide
  _df = _df / (np.sqrt(abs(_df@H@_df)))
/home/elias/Desktop/PROJECTS/SPP/SPP/Solvers/Ellipsoids.py:77: RuntimeWarning: invalid value encountered in matmul
  x = x - 1/(n+1) * H @ _df
/home/elias/Desktop/PROJECTS/SPP/SPP/Solvers/Ellipsoids.py:78: RuntimeWarning: invalid value encountered in matmul
  H = n**2/(n**2 - 1)*(H - (2 / (n + 1)) * (H @ np.outer(_df, _df) @ H))


In [10]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$x, y - FGM+Ellipsoids	$x, y - FGM+Dichotomy	$
False		$True		$True		$True		$True		$False		$
40.001		$0.406		$0.028		$0.158		$40.008		$40.005		$
4.615		$4.615		$1510.792		$4.615		$4.615		$4.615		$


In [11]:
eps = 1e-9
history, stop_cond_args = start_comparison(N = N, M = M,beta = 1e-6, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [12]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$x, y - FGM+Ellipsoids	$x, y - FGM+Dichotomy	$
False		$True		$True		$True		$True		$True		$
40.012		$1.071		$0.393		$0.425		$40.001		$40.006		$
4.615		$4.615		$161.952		$4.615		$4.615		$4.615		$


### N = 1000

In [13]:
N = 1000

In [14]:
eps = 1e-3
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

/home/elias/Desktop/PROJECTS/SPP/SPP/Solvers/Ellipsoids.py:76: RuntimeWarning: invalid value encountered in true_divide
  _df = _df / (np.sqrt(abs(_df@H@_df)))


In [15]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$x, y - FGM+Ellipsoids	$x, y - FGM+Dichotomy	$
False		$True		$True		$True		$True		$True		$
40.009		$0.197		$0.048		$0.029		$4.966		$1.966		$
6.909		$6.909		$6.910		$6.909		$6.909		$6.909		$


In [16]:
eps = 1e-6
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [17]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$x, y - FGM+Ellipsoids	$x, y - FGM+Dichotomy	$
False		$True		$True		$True		$True		$False		$
40.008		$0.706		$0.264		$0.153		$40.006		$40.006		$
6.909		$6.909		$6.909		$6.909		$6.909		$6.909		$


In [18]:
eps = 1e-9
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [19]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$x, y - FGM+Ellipsoids	$x, y - FGM+Dichotomy	$
False		$True		$True		$True		$True		$True		$
40.009		$1.435		$1.096		$0.390		$40.016		$40.003		$
6.909		$6.909		$6.909		$6.909		$6.909		$6.909		$


## M= 3

In [20]:
M = 3

### N = 100

In [21]:
N = 100

In [22]:
eps = 1e-3
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [23]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$x, y - FGM+Ellipsoids	$x, y - FGM+Dichotomy	$
False		$True		$True		$True		$True		$True		$
40.001		$0.226		$0.250		$0.034		$12.387		$25.773		$
4.615		$4.616		$4.615		$4.615		$4.615		$4.615		$


In [24]:
eps = 1e-6
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [25]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$x, y - FGM+Ellipsoids	$x, y - FGM+Dichotomy	$
False		$True		$True		$True		$True		$True		$
40.006		$0.894		$0.843		$0.220		$40.026		$40.011		$
4.615		$4.615		$4.615		$4.615		$4.615		$4.615		$


In [26]:
eps = 1e-9
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [27]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$x, y - FGM+Ellipsoids	$x, y - FGM+Dichotomy	$
False		$True		$True		$True		$True		$True		$
40.008		$1.883		$4.382		$0.538		$40.032		$40.006		$
4.615		$4.615		$4.615		$4.615		$4.615		$4.615		$


### N = 1000

In [28]:
N = 1000

In [29]:
eps = 1e-3
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [30]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$x, y - FGM+Ellipsoids	$x, y - FGM+Dichotomy	$
False		$True		$True		$True		$True		$True		$
40.003		$0.473		$0.248		$0.034		$13.054		$21.366		$
6.909		$6.909		$6.909		$6.909		$6.909		$6.909		$


In [31]:
eps = 1e-6
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [32]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$x, y - FGM+Ellipsoids	$x, y - FGM+Dichotomy	$
False		$True		$True		$True		$True		$True		$
40.002		$1.677		$1.387		$0.224		$40.018		$40.013		$
6.909		$6.909		$6.909		$6.909		$6.909		$6.909		$


In [33]:
eps = 1e-9
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [34]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$x, y - FGM+Ellipsoids	$x, y - FGM+Dichotomy	$
False		$True		$True		$True		$True		$True		$
40.007		$3.596		$7.450		$0.604		$40.024		$40.004		$
6.909		$6.909		$6.909		$6.909		$6.909		$6.909		$


## M= 4

In [35]:
M = 4

### N = 100

In [36]:
N = 100

In [37]:
eps = 1e-3
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [38]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$x, y - FGM+Ellipsoids	$x, y - FGM+Dichotomy	$
False		$True		$True		$True		$True		$True		$
40.005		$0.388		$0.164		$0.058		$26.134		$67.907		$
4.615		$4.616		$18.416		$4.615		$4.615		$4.615		$


In [39]:
eps = 1e-6
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [40]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$x, y - FGM+Ellipsoids	$x, y - FGM+Dichotomy	$
False		$True		$True		$True		$True		$True		$
40.001		$1.555		$3.860		$0.265		$40.047		$40.034		$
4.615		$4.615		$4.615		$4.615		$4.615		$4.615		$


In [41]:
eps = 1e-9
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [42]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$x, y - FGM+Ellipsoids	$x, y - FGM+Dichotomy	$
False		$True		$True		$True		$True		$True		$
40.004		$2.841		$10.213		$0.749		$40.071		$40.016		$
4.615		$4.615		$4.615		$4.615		$4.615		$4.615		$


### N = 1000

In [43]:
N = 1000

In [44]:
eps = 1e-3
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [45]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$x, y - FGM+Ellipsoids	$x, y - FGM+Dichotomy	$
False		$True		$True		$True		$True		$True		$
40.006		$0.903		$0.236		$0.043		$25.905		$173.770		$
6.909		$6.909		$6.909		$6.909		$6.909		$6.909		$


In [46]:
eps = 1e-6
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [47]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$x, y - FGM+Ellipsoids	$x, y - FGM+Dichotomy	$
False		$True		$True		$True		$True		$True		$
40.016		$3.240		$9.569		$0.444		$40.013		$40.003		$
6.909		$6.909		$6.909		$6.909		$6.909		$6.909		$


In [48]:
eps = 1e-9
history, stop_cond_args = start_comparison(N = N, M = M, eps = eps, time_max = 40)
alpha_, B, c, beta = stop_cond_args

In [49]:
parse_results(history, alpha_, B, c, beta, keys = history.keys())

y, x - FGM+FGM	$y, x - Ellipsoids+FGM	$y, x - Dichotomy+FGM	$x, y - FGM+FGM	$x, y - FGM+Ellipsoids	$x, y - FGM+Dichotomy	$
False		$True		$True		$True		$True		$True		$
40.012		$6.269		$23.327		$0.938		$40.072		$40.032		$
6.909		$6.909		$6.909		$6.909		$6.909		$6.909		$
